# `pycapse` usage example

In this notebook, we'll show how to use `pycapse` to predict the correlation functions.

## Loading relevant libraries

The first step is to import the relevant libraries

In [ ]:
import numpy as np
import pycapse as pc
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

## Loading the trained emulator
After succesfully loading `pybora`, we need to load the trained emulator

In [ ]:
emu_tt = pc.load_emu("emulators_Planck_idm_dr_n4_nonlinear_halofit_TT.bson")
emu_ee = pc.load_emu("emulators_Planck_idm_dr_n4_nonlinear_halofit_EE.bson")
emu_te = pc.load_emu("emulators_Planck_idm_dr_n4_nonlinear_halofit_TE.bson")
emu_pp = pc.load_emu("emulators_Planck_idm_dr_n4_nonlinear_halofit_PP.bson")

## Running the code
Now we can actually run the code!

In [ ]:
lgrid_tt = pc.get_lgrid(emu_tt)
lgrid_ee = pc.get_lgrid(emu_ee)
lgrid_te = pc.get_lgrid(emu_te)
lgrid_pp = pc.get_lgrid(emu_pp)
cosmo = np.array([2.8472231, 1.0196047, 0.72768, 0.0206263, 0.1492, 0.068604, 0.289396, 4.])
test_TT = pc.compute_Cl(cosmo, emu_tt)
test_TE = pc.compute_Cl(cosmo, emu_te)
test_EE = pc.compute_Cl(cosmo, emu_ee)
test_PP = pc.compute_Cl(cosmo, emu_pp)
%timeit pc.compute_Cl(cosmo, emu_tt)

Let's plot the result of our calculations.

In [ ]:
plt.plot(lgrid_tt, lgrid_tt**2*test_TT, label = "TT")
plt.xlabel(r'$\ell$')
plt.ylabel(r'$\ell(\ell+1)C_\ell^2$')
plt.legend()

In [ ]:
plt.plot(lgrid_ee, lgrid_ee**2*test_EE, label = "EE")
plt.xlabel(r'$\ell$')
plt.ylabel(r'$\ell(\ell+1)C_\ell^2$')
plt.legend()

In [ ]:
plt.plot(lgrid_te, lgrid_te**2*test_TE, label = "TE")
plt.xlabel(r'$\ell$')
plt.ylabel(r'$\ell(\ell+1)C_\ell^2$')
plt.legend()

In [ ]:
plt.plot(lgrid_pp, (lgrid_pp*(lgrid_pp+1))**2*test_PP, label = "PP")
plt.xlabel(r'$\ell$')
plt.ylabel(r'$(\ell(\ell+1))^2C_\ell^2$')
plt.legend()